In [1]:
import pandas as pd
import pandas_profiling as pprf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle

In [2]:
# https://www.kaggle.com/thegurus/spanish-high-speed-rail-system-ticket-pricing

path = r'E:\datasets\spanish-high-speed-rail-system-ticket-pricing\renfe.csv'
df = pd.read_csv(path)
df = shuffle(df)
# Run these lines only one time
#profile_report = pprf.ProfileReport(df) 
#profile_report.to_file(r'C:\spanish_rail_system_profile_report.html')
df.head()

,Unnamed: 0,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
574755,574755,2019-04-23 11:17:50,BARCELONA,MADRID,2019-05-24 17:00:00,2019-05-24 19:45:00,AVE,85.10,Turista,Promo
1232196,1232196,2019-05-03 13:45:15,MADRID,BARCELONA,2019-05-05 19:00:00,2019-05-05 21:30:00,AVE,120.50,Turista Plus,Promo
990069,990069,2019-04-29 01:32:04,PONFERRADA,MADRID,2019-05-26 17:15:00,2019-05-26 22:14:00,MD-AVE,35.15,Turista con enlace,Promo +
313293,313293,2019-04-21 13:38:13,MADRID,VALENCIA,2019-05-21 17:40:00,2019-05-21 19:20:00,AVE,57.75,Turista,Promo
60093,60093,2019-04-19 16:17:43,SEVILLA,MADRID,2019-05-10 18:45:00,2019-05-10 21:17:00,AVE,76.30,Turista,Flexible


In [3]:
df = df.dropna()
df.head()

,Unnamed: 0,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
574755,574755,2019-04-23 11:17:50,BARCELONA,MADRID,2019-05-24 17:00:00,2019-05-24 19:45:00,AVE,85.10,Turista,Promo
1232196,1232196,2019-05-03 13:45:15,MADRID,BARCELONA,2019-05-05 19:00:00,2019-05-05 21:30:00,AVE,120.50,Turista Plus,Promo
990069,990069,2019-04-29 01:32:04,PONFERRADA,MADRID,2019-05-26 17:15:00,2019-05-26 22:14:00,MD-AVE,35.15,Turista con enlace,Promo +
313293,313293,2019-04-21 13:38:13,MADRID,VALENCIA,2019-05-21 17:40:00,2019-05-21 19:20:00,AVE,57.75,Turista,Promo
60093,60093,2019-04-19 16:17:43,SEVILLA,MADRID,2019-05-10 18:45:00,2019-05-10 21:17:00,AVE,76.30,Turista,Flexible


In [4]:
df.index = range(len(df))
df.head()

,Unnamed: 0,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
0,574755,2019-04-23 11:17:50,BARCELONA,MADRID,2019-05-24 17:00:00,2019-05-24 19:45:00,AVE,85.10,Turista,Promo
1,1232196,2019-05-03 13:45:15,MADRID,BARCELONA,2019-05-05 19:00:00,2019-05-05 21:30:00,AVE,120.50,Turista Plus,Promo
2,990069,2019-04-29 01:32:04,PONFERRADA,MADRID,2019-05-26 17:15:00,2019-05-26 22:14:00,MD-AVE,35.15,Turista con enlace,Promo +
3,313293,2019-04-21 13:38:13,MADRID,VALENCIA,2019-05-21 17:40:00,2019-05-21 19:20:00,AVE,57.75,Turista,Promo
4,60093,2019-04-19 16:17:43,SEVILLA,MADRID,2019-05-10 18:45:00,2019-05-10 21:17:00,AVE,76.30,Turista,Flexible


In [5]:
target = df['price'].values
df = df.drop(['Unnamed: 0', 'price'], axis=1)
df.head()

,insert_date,origin,destination,start_date,end_date,train_type,train_class,fare
0,2019-04-23 11:17:50,BARCELONA,MADRID,2019-05-24 17:00:00,2019-05-24 19:45:00,AVE,Turista,Promo
1,2019-05-03 13:45:15,MADRID,BARCELONA,2019-05-05 19:00:00,2019-05-05 21:30:00,AVE,Turista Plus,Promo
2,2019-04-29 01:32:04,PONFERRADA,MADRID,2019-05-26 17:15:00,2019-05-26 22:14:00,MD-AVE,Turista con enlace,Promo +
3,2019-04-21 13:38:13,MADRID,VALENCIA,2019-05-21 17:40:00,2019-05-21 19:20:00,AVE,Turista,Promo
4,2019-04-19 16:17:43,SEVILLA,MADRID,2019-05-10 18:45:00,2019-05-10 21:17:00,AVE,Turista,Flexible


In [6]:
def delta_in_time(start_dates:pd.Series, end_dates:pd.Series):
    # Warning! end_dates must be "greater" than start_dates
    start_dates = pd.to_datetime(start_dates)
    end_dates = pd.to_datetime(end_dates)
    
    return list(i.seconds / 60 for i in (end_dates - start_dates))

In [7]:
df['duration'] = delta_in_time(df['start_date'], df['end_date'])
df.head()

,insert_date,origin,destination,start_date,end_date,train_type,train_class,fare,duration
0,2019-04-23 11:17:50,BARCELONA,MADRID,2019-05-24 17:00:00,2019-05-24 19:45:00,AVE,Turista,Promo,165.0
1,2019-05-03 13:45:15,MADRID,BARCELONA,2019-05-05 19:00:00,2019-05-05 21:30:00,AVE,Turista Plus,Promo,150.0
2,2019-04-29 01:32:04,PONFERRADA,MADRID,2019-05-26 17:15:00,2019-05-26 22:14:00,MD-AVE,Turista con enlace,Promo +,299.0
3,2019-04-21 13:38:13,MADRID,VALENCIA,2019-05-21 17:40:00,2019-05-21 19:20:00,AVE,Turista,Promo,100.0
4,2019-04-19 16:17:43,SEVILLA,MADRID,2019-05-10 18:45:00,2019-05-10 21:17:00,AVE,Turista,Flexible,152.0


In [8]:
def get_time_of_day(date):
    hour = int(date.split(' ')[1].split(':')[0])
    
    if hour < 4:
        return 'night'
    elif hour < 12:
        return 'morning'
    elif hour < 17:
        return 'day'
    else:
        return 'evening'
    
df['time_of_day'] = df['insert_date'].apply(get_time_of_day)
df.head()

,insert_date,origin,destination,start_date,end_date,train_type,train_class,fare,duration,time_of_day
0,2019-04-23 11:17:50,BARCELONA,MADRID,2019-05-24 17:00:00,2019-05-24 19:45:00,AVE,Turista,Promo,165.0,morning
1,2019-05-03 13:45:15,MADRID,BARCELONA,2019-05-05 19:00:00,2019-05-05 21:30:00,AVE,Turista Plus,Promo,150.0,day
2,2019-04-29 01:32:04,PONFERRADA,MADRID,2019-05-26 17:15:00,2019-05-26 22:14:00,MD-AVE,Turista con enlace,Promo +,299.0,night
3,2019-04-21 13:38:13,MADRID,VALENCIA,2019-05-21 17:40:00,2019-05-21 19:20:00,AVE,Turista,Promo,100.0,day
4,2019-04-19 16:17:43,SEVILLA,MADRID,2019-05-10 18:45:00,2019-05-10 21:17:00,AVE,Turista,Flexible,152.0,day


In [9]:
def get_time_of_year(date):
    month = int(date.split(' ')[0].split('-')[1])
    
    if (month < 3) or (month == 12):
        return 'winter'
    elif month < 6:
        return 'spring'
    elif month < 9:
        return 'summer'
    else:
        return 'autumn'
    
df['time_of_year'] = df['insert_date'].apply(get_time_of_year)    
df.head()

,insert_date,origin,destination,start_date,end_date,train_type,train_class,fare,duration,time_of_day,time_of_year
0,2019-04-23 11:17:50,BARCELONA,MADRID,2019-05-24 17:00:00,2019-05-24 19:45:00,AVE,Turista,Promo,165.0,morning,spring
1,2019-05-03 13:45:15,MADRID,BARCELONA,2019-05-05 19:00:00,2019-05-05 21:30:00,AVE,Turista Plus,Promo,150.0,day,spring
2,2019-04-29 01:32:04,PONFERRADA,MADRID,2019-05-26 17:15:00,2019-05-26 22:14:00,MD-AVE,Turista con enlace,Promo +,299.0,night,spring
3,2019-04-21 13:38:13,MADRID,VALENCIA,2019-05-21 17:40:00,2019-05-21 19:20:00,AVE,Turista,Promo,100.0,day,spring
4,2019-04-19 16:17:43,SEVILLA,MADRID,2019-05-10 18:45:00,2019-05-10 21:17:00,AVE,Turista,Flexible,152.0,day,spring


In [10]:
df = df.drop(['insert_date', 'start_date', 'end_date'], axis=1)
df.head()

,origin,destination,train_type,train_class,fare,duration,time_of_day,time_of_year
0,BARCELONA,MADRID,AVE,Turista,Promo,165.0,morning,spring
1,MADRID,BARCELONA,AVE,Turista Plus,Promo,150.0,day,spring
2,PONFERRADA,MADRID,MD-AVE,Turista con enlace,Promo +,299.0,night,spring
3,MADRID,VALENCIA,AVE,Turista,Promo,100.0,day,spring
4,SEVILLA,MADRID,AVE,Turista,Flexible,152.0,day,spring


In [11]:
df = pd.get_dummies(df)
df.values.shape

(2269090, 44)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2269090 entries, 0 to 2269089
Data columns (total 44 columns):
duration                          float64
origin_BARCELONA                  uint8
origin_MADRID                     uint8
origin_PONFERRADA                 uint8
origin_SEVILLA                    uint8
origin_VALENCIA                   uint8
destination_BARCELONA             uint8
destination_MADRID                uint8
destination_PONFERRADA            uint8
destination_SEVILLA               uint8
destination_VALENCIA              uint8
train_type_ALVIA                  uint8
train_type_AV City                uint8
train_type_AVE                    uint8
train_type_AVE-LD                 uint8
train_type_AVE-MD                 uint8
train_type_AVE-TGV                uint8
train_type_INTERCITY              uint8
train_type_LD                     uint8
train_type_LD-MD                  uint8
train_type_MD                     uint8
train_type_MD-AVE                 uint8
train

In [13]:
samples = normalize(df.values)
samples[0]

array([0.99987147, 0.00605983, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00605983, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00605983, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00605983,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00605983, 0.        , 0.        ,
       0.        , 0.00605983, 0.        , 0.00605983])

In [14]:
def get_model():
    
    model = Sequential()
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='linear'))
    
    model.compile(optimizer='adam', 
                  loss='mse', 
                  metrics=['mae'])
    
    return model

In [15]:
target.shape, samples.shape

((2269090,), (2269090, 44))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(samples, target, test_size=.1)
X_train.shape, y_train.shape

((2042181, 44), (2042181,))

In [17]:
model = get_model()

history = model.fit(X_train, 
                    y_train, 
                    epochs=50, 
                    batch_size=20000, 
                    validation_data=(X_test, y_test))
history

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 2042181 samples, validate on 226909 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
2042181/2042181 [==============================] - 24s 12us/sample - loss: 3586.6346 - mean_absolute_error: 52.9074 - val_loss: 794.1627 - val_mean_absolute_error: 22.1181
Epoch 2/50
2042181/2042181 [==============================] - 14s 7us/sample - loss: 734.1044 - mean_absolute_error: 21.6457 - val_loss: 659.7349 - val_mean_absolute_error: 20.6704
Epoch 3/50
2042181/2042181 [==============================] - 14s 7us/sample - loss: 717.0371 - mean_absolute_error: 21.3807 - val_loss: 653.4646 - val_mean_absolute_error: 20.5496
Epoch 4/50
2042181/2042181 [==============================] - 15s 7us/sample - loss: 705.7504 - mean_absolute_error: 21

2042181/2042181 [==============================] - 13s 6us/sample - loss: 146.9383 - mean_absolute_error: 8.9522 - val_loss: 105.7505 - val_mean_absolute_error: 7.2445
Epoch 40/50
2042181/2042181 [==============================] - 13s 6us/sample - loss: 145.7057 - mean_absolute_error: 8.9074 - val_loss: 106.0026 - val_mean_absolute_error: 7.2417
Epoch 41/50
2042181/2042181 [==============================] - 13s 6us/sample - loss: 144.8803 - mean_absolute_error: 8.8774 - val_loss: 105.5369 - val_mean_absolute_error: 7.2288
Epoch 42/50
2042181/2042181 [==============================] - 14s 7us/sample - loss: 144.0727 - mean_absolute_error: 8.8456 - val_loss: 107.1515 - val_mean_absolute_error: 7.2967
Epoch 43/50
2042181/2042181 [==============================] - 14s 7us/sample - loss: 143.2409 - mean_absolute_error: 8.8143 - val_loss: 106.5483 - val_mean_absolute_error: 7.2510
Epoch 44/50
2042181/2042181 [==============================] - 16s 8us/sample - loss: 142.5562 - mean_absolute_e